In [2]:
import numpy as np
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from lpne.models import DcsfaNmf

MODEL_FILE = "/hpc/home/mk423/Anxiety/FullDataWork/Models/Final_mt_Model_500_epochs.pt"
DATA_PATH = "/work/mk423/Anxiety/"
PROJECT_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Projections/"
dzp_file = DATA_PATH + "DZP_data.pkl"
proj_file = PROJECT_PATH + "DZP_projection.csv"
mean_file = PROJECT_PATH + "DZP_mean_scores.csv"
model = torch.load(MODEL_FILE,map_location="cpu")
model.device="cpu"


FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
FEATURE_WEIGHT = [10,1,1]

import os, sys
umc_data_tools_path = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
sys.path.append(umc_data_tools_path)
import umc_data_tools as umc_dt

def get_3_net_aucs(s,y,y_group=None):
    
    auc_mean_list = []
    auc_stderr_list =[]
    
    if y_group is None:
        y_group = np.ones(s.shape[0])
        
    for i in range(3):
        auc_dict = umc_dt.lpne_auc(y,y,y_group,s[:,i].reshape(-1,1),mannWhitneyU=True)
        mean = np.mean([auc_dict[key][0] for key in auc_dict.keys() if key != "auc_method"])
        stderr = np.std([auc_dict[key][0] for key in auc_dict.keys() if key != "auc_method"]) / np.sqrt(len(auc_dict.keys()))
        
        auc_mean_list.append(mean)
        auc_stderr_list.append(stderr)
        
    return auc_mean_list, auc_stderr_list

def get_3_net_auc_dict(s,y,y_group=None):
    auc_dict_list = []
    
    if y_group is None:
        y_group = np.ones(s.shape[0])
        
    for i in range(3):
        auc_dict = umc_dt.lpne_auc(y,y,y_group,s[:,i].reshape(-1,1),mannWhitneyU=True)
        auc_dict_list.append(auc_dict)
        
    return auc_dict_list

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [3]:
dzp_data_dict = pickle.load(open(dzp_file,'rb'))

X = dzp_data_dict['X']
X[:,:448] = X[:,:448]*10 #Scale the Power Features
y_mouse = dzp_data_dict['y_mouse']
y_time = dzp_data_dict['y_time']
y_expDate = dzp_data_dict['y_expDate']
y_dzp = dzp_data_dict['y_dzp']

In [4]:
y_pred = model.predict_proba(X)
s = model.project(X)
dzp_auc = model.score(X,y_dzp,y_mouse,return_dict=True)
print(dzp_auc)

dzp_auc_list = [dzp_auc[key] for key in dzp_auc.keys() if key != "Mouse39114"]
mean = np.mean(dzp_auc_list)
sterr = np.std(dzp_auc_list) / np.sqrt(len(dzp_auc_list))

print("DZP AUC {} +/- {} (n={})".format(mean,sterr,len(dzp_auc_list)))

{'Mouse04191': [0.4262415217304699], 'Mouse04201': [0.4770587872352452], 'Mouse04203': [0.4168375943430349], 'Mouse04215': [0.3993645805667209], 'Mouse39114': [nan], 'Mouse39121': [0.4458473160954862], 'Mouse39124': [0.581818012277204], 'Mouse39133': [0.5873825898584559]}
DZP AUC 0.47636434315808807 +/- 0.027241172738363212 (n=7)


In [5]:
dzp_auc_3_net = get_3_net_auc_dict(s,y_dzp,y_mouse)

Mouse  Mouse39114  has only one class - AUC cannot be calculated
n_positive samples  3148
n_negative samples  0
Mouse  Mouse39114  has only one class - AUC cannot be calculated
n_positive samples  3148
n_negative samples  0
Mouse  Mouse39114  has only one class - AUC cannot be calculated
n_positive samples  3148
n_negative samples  0


In [6]:
results_dict = {
    "net 1 scores":s[:,0],
    "net 2 scores":s[:,1],
    "net 3 scores":s[:,2],
    "DZP":y_dzp,
    "mouse":y_mouse,
    "time":y_time,
    "expDate":y_expDate,
    "dzp vs sal auc": [dzp_auc[mouse][0] for mouse in y_mouse],
    "net 1 auc":[dzp_auc_3_net[0][mouse][0] for mouse in y_mouse],
    "net 2 auc":[dzp_auc_3_net[1][mouse][0] for mouse in y_mouse],
    "net 3 auc":[dzp_auc_3_net[2][mouse][0] for mouse in y_mouse],
}

df = pd.DataFrame.from_dict(results_dict)
df.to_csv(proj_file)
df.head()


,net 1 scores,net 2 scores,net 3 scores,DZP,mouse,time,expDate,dzp vs sal auc,net 1 auc,net 2 auc,net 3 auc
0,0.077796,0.077701,0.049457,1.0,Mouse04191,1,100121,0.426242,0.348761,0.459872,0.544677
1,0.120851,0.141939,0.037657,1.0,Mouse04191,2,100121,0.426242,0.348761,0.459872,0.544677
2,0.104863,0.153451,0.081454,1.0,Mouse04191,3,100121,0.426242,0.348761,0.459872,0.544677
3,0.133968,0.190871,0.059753,1.0,Mouse04191,4,100121,0.426242,0.348761,0.459872,0.544677
4,0.102102,0.242141,0.088506,1.0,Mouse04191,5,100121,0.426242,0.348761,0.459872,0.544677


In [8]:
mouse_list = []
avg_score_list = []
avg_drug_score_list = []
avg_sal_score_list = []

for mouse in np.unique(y_mouse):
    mouse_mask = y_mouse==mouse
    drug_mouse_mask = np.logical_and(mouse_mask,y_dzp)
    sal_mouse_mask = np.logical_and(mouse_mask,1-y_dzp)
    
    avg_score = np.mean(s[mouse_mask,:3],axis=0)
    avg_drug_score = np.mean(s[drug_mouse_mask,:3],axis=0)
    avg_sal_score = np.mean(s[sal_mouse_mask,:3],axis=0)
    
    mouse_list.append(mouse)
    avg_score_list.append(avg_score)
    avg_drug_score_list.append(avg_drug_score)
    avg_sal_score_list.append(avg_sal_score)
    

avg_score_list = np.array(avg_score_list)
avg_drug_score_list = np.array(avg_drug_score_list)
avg_sal_score_list = np.array(avg_sal_score_list)

coeffs = np.abs(model.classifier[0].weight[0].detach().cpu().numpy())

mag_score_impact = avg_score_list*coeffs
mag_drug_impact = avg_drug_score_list*coeffs
mag_sal_impact = avg_sal_score_list*coeffs

avg_score_impact = mag_score_impact / np.sum(mag_score_impact,axis=1).reshape(-1,1)
avg_drug_impact = mag_drug_impact / np.sum(mag_drug_impact,axis=1).reshape(-1,1)
avg_sal_impact = mag_sal_impact / np.sum(mag_sal_impact,axis=1).reshape(-1,1)

scores_dict = {
    "mouse":mouse_list,
    
    "composite avgScore":np.sum(mag_score_impact,axis=1),
    "composite avgDrugScore":np.sum(mag_drug_impact,axis=1),
    "composite avgSalScore":np.sum(mag_sal_impact,axis=1),
    
    "net 1 avgScore":avg_score_list[:,0],
    "net 1 avgDrugScore":avg_drug_score_list[:,0],
    "net 1 avgSalScore":avg_sal_score_list[:,0],
    
    "net 2 avgScore":avg_score_list[:,1],
    "net 2 avgDrugScore":avg_drug_score_list[:,1],
    "net 2 avgSalScore":avg_sal_score_list[:,1],
    
    "net 3 avgScore":avg_score_list[:,2],
    "net 3 avgDrugScore":avg_drug_score_list[:,2],
    "net 3 avgSalScore":avg_sal_score_list[:,2],
    
    "net 1 avgImpact":avg_score_list[:,0],
    "net 1 avgDrugImpact":avg_drug_score_list[:,0],
    "net 1 avgSalImpact":avg_sal_score_list[:,0],
    
    "net 2 avgImpact":avg_score_list[:,1],
    "net 2 avgDrugImpact":avg_drug_score_list[:,1],
    "net 2 avgSalImpact":avg_sal_score_list[:,1],
    
    "net 3 avgImpact":avg_score_list[:,2],
    "net 3 avgDrugImpact":avg_drug_score_list[:,2],
    "net 3 avgSalImpact":avg_sal_score_list[:,2],
}

df_mean = pd.DataFrame.from_dict(scores_dict)
df_mean.to_csv(mean_file)
df_mean.head()

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


,mouse,composite avgScore,composite avgDrugScore,composite avgSalScore,net 1 avgScore,net 1 avgDrugScore,net 1 avgSalScore,net 2 avgScore,net 2 avgDrugScore,net 2 avgSalScore,...,net 3 avgSalScore,net 1 avgImpact,net 1 avgDrugImpact,net 1 avgSalImpact,net 2 avgImpact,net 2 avgDrugImpact,net 2 avgSalImpact,net 3 avgImpact,net 3 avgDrugImpact,net 3 avgSalImpact
0,Mouse04191,2.622164,2.510283,2.737094,0.098896,0.092194,0.105781,0.097061,0.094202,0.099997,...,0.046561,0.098896,0.092194,0.105781,0.097061,0.094202,0.099997,0.048599,0.050584,0.046561
1,Mouse04201,3.199795,3.170409,3.230079,0.083039,0.082672,0.083417,0.140341,0.138447,0.142292,...,0.055996,0.083039,0.082672,0.083417,0.140341,0.138447,0.142292,0.064224,0.072207,0.055996
2,Mouse04203,2.654890,2.494308,2.720083,0.101527,0.099643,0.102292,0.097974,0.089333,0.101482,...,0.036062,0.101527,0.099643,0.102292,0.097974,0.089333,0.101482,0.037154,0.039845,0.036062
3,Mouse04215,3.754377,3.508299,3.862508,0.071849,0.072664,0.071492,0.180795,0.164986,0.187741,...,0.044156,0.071849,0.072664,0.071492,0.180795,0.164986,0.187741,0.049942,0.063108,0.044156
4,Mouse39114,3.161655,3.161655,NaN,0.091762,0.091762,NaN,0.133242,0.133242,NaN,...,NaN,0.091762,0.091762,NaN,0.133242,0.133242,NaN,0.057023,0.057023,NaN
